In [1]:
library(fastDummies)
library(cplm)
library(tweedie)
library(statmod)
library(TDboost)
library(mgcv)
library(MASS)
library(glmnet)
source("S_U2.R")
options(warn=-1)

Loading required package: coda

Loading required package: Matrix

Loading required package: splines

Loading required package: lattice

Loaded TDboost 1.2


Loading required package: nlme


Attaching package: 'nlme'


The following objects are masked from 'package:cplm':

    fixef, ranef, VarCorr


This is mgcv 1.8-38. For overview type 'help("mgcv-package")'.

Loaded glmnet 4.1-2



In [2]:
D_beta<-matrix(nrow=16,ncol=100)
A_beta<-matrix(nrow=16,ncol=100)
LASSO_beta<-matrix(nrow=16,ncol=100)
B_beta<-matrix(nrow=16,ncol=100)
nc=1

In [3]:
while(nc<=100){
    x<-matrix( rnorm(1000*10,mean=0,sd=1),1000, 5) 
    colnames(x)=c('x1','x2','x3','x4','x5')
    X<-as.data.frame(x)
    b_t<-c(-0.3,0.5,-0.5,0.5,0,0,-.8,-.8,-0.8)
    design_x <- model.matrix(~.+x1*x2+x1*x3+x2*x3,data=X)
    mu<-exp(design_x%*%b_t)
    y_sim<- rTweedie(mu,p=1.5,phi=5)
    dd<-as.data.frame(cbind(design_x[,2:6],y_sim))
    sam <- sample(rep(1:2,len=1000))
    sim_train<-dd[sam==1,]
    sim_test<-dd[sam==2,]
    sim_train<-as.data.frame(sim_train)
    sim_test<-as.data.frame(sim_test)
    colnames(sim_train)[6]='y'
    colnames(sim_test)[6]='y'
    fit_0<-glm(y~.^2,data=sim_train,family=tweedie(link.power=0,var.power=1.5),control=list(maxit=500))
    beta_0_t<-c(fit_0$coefficients[1:6])
    index=combn(c(1:5),2)
    int_coef<-fit_0$coefficients[-(1:6)]
    r_0_t<-c()
    for (i in 1:length(int_coef)){
        tmp_coef<-int_coef[i]/(beta_0_t[index[1,i]+1]*beta_0_t[index[2,i]+1])
        r_0_t<-c(r_0_t,tmp_coef)
    }
    X<-model.matrix(~.^2-1,data=sim_train[,1:5])
    qw=cv.glmnet(x=X,y=sim_train$y,family=tweedie(link.power=0,var.power=1.5),control=list(maxit=500),nlambda=10)
    LASSO_beta[,nc]<-coef(qw, s=qw$lambda.min)[,1]
    as=cv.glmnet(x=X[,1:5],y=sim_train$y,family=tweedie(link.power=0,var.power=1.5),control=list(maxit=500),nlambda=10)
    df=cv.glmnet(x=X[,-c(1:5)],y=sim_train$y,family=tweedie(link.power=0,var.power=1.5),control=list(maxit=500),nlambda=10,nfolds=5)
    r_lamb<-df$lambda
    beta_lamb<-as$lambda
    n_lamb<-length(beta_lamb)
    power=1.5
    final_result1=matrix(nrow=n_lamb,ncol=n_lamb)
    final_result2=matrix(nrow=n_lamb,ncol=n_lamb)
    final_result3=matrix(nrow=n_lamb,ncol=n_lamb)
    for (i in 1:n_lamb){
        for (j in 1:n_lamb){
            coef_result<-shim_lasso_update_2(sim_train[,1:5],sim_train$y,power,lamb_r=r_lamb[i],lamb_b=beta_lamb[j],r_0_t,beta_0_t)
            fitted<-ts_fitted(sim_test[,1:5],coef_result$'beta',coef_result$'r')
            final_result1[i,j]=mean(tweedie.dev(sim_test$y,fitted,1.5))
            fitted<-ts_fitted(sim_train[,1:5],coef_result$'beta',coef_result$'r')
            if (is.infinite(mean(tweedie.dev(sim_train$y,fitted,1.5),na.rm=T))){
                    final_result2[i,j]<-999999999
                    final_result3[i,j]<-999999999
                        }
            else{
                den<-dtweedie(sim_train$y,mu=fitted+0.0000001,phi=sum(tweedie.dev(sim_train$y,fitted,1.5),na.rm=T)/length(sim_train$y),power=1.5)
                df<-length(which(r_beta(coef_result$'beta',coef_result$'r')!=0))
                final_result2[i,j]<--2*sum( log(den))+2*(df)
                final_result3[i,j]<--2*sum( log(den))+log(length(sim_train$y))*(df)
                }
    }
        }
    index1<-which(final_result1==min(final_result1,na.rm=T), arr.ind = TRUE)[1,]
    index2<-which(final_result2==min(final_result2,na.rm=T), arr.ind = TRUE)[1,]
    index3<-which(final_result3==min(final_result3,na.rm=T), arr.ind = TRUE)[1,]
    coef_result<-shim_lasso_update_2(sim_train[,1:5],sim_train$y,power,lamb_r=r_lamb[index1[1]],lamb_b=beta_lamb[index1[2]],r_0_t,beta_0_t)
    D_beta[,nc]<-r_beta(coef_result$beta,coef_result$r)
    coef_result<-shim_lasso_update_2(sim_train[,1:5],sim_train$y,power,lamb_r=r_lamb[index2[1]],lamb_b=beta_lamb[index2[2]],r_0_t,beta_0_t)
    A_beta[,nc]<-r_beta(coef_result$beta,coef_result$r)
    coef_result<-shim_lasso_update_2(sim_train[,1:5],sim_train$y,power,lamb_r=r_lamb[index3[1]],lamb_b=beta_lamb[index3[2]],r_0_t,beta_0_t)
    B_beta[,nc]<-r_beta(coef_result$beta,coef_result$r)
    nc=nc+1
}

In [8]:
which(is.na(A_beta[1,]))

[1]   9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27
[20]  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46
[39]  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65
[58]  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84
[77]  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100

In [7]:
write.matrix(A_beta,file="simulation444A.csv")
write.matrix(D_beta,file="simulation444D.csv")
write.matrix(B_beta,file="simulation444B.csv")
write.matrix(LASSO_beta,file="simulation444L.csv")

In [8]:
check_function<-function(x){
    xlen<-length(x)
    return(all(x[1:4]!=0)&all(x[5:6]==0)&all(x[7:8]!=0)&all(x[9:10]==0)&all(x[11]!=0)&all(x[12:xlen]==0))
}

check_function2<-function(x){
    xlen<-length(x)
    return(all(x[1:4]!=0)&all(x[7:8]!=0)&x[11]!=0)
}

distance_function<-function(x,b=c(-0.3,0.5,-0.5,0.5,0,0,-0.8,-0.8,0,0,-0.8,rep(0,5))){
    return(sqrt(mean((x-b)^2)))
}

distance_function_2<-function(x,b=c(-0.3,0.5,-0.5,0.5,0,0,-0.8,-0.8,0,0,-0.8,rep(0,5))){
    return(mean(abs(x-b)))
}

In [5]:
length(which(apply(A_beta,2,check_function)))
length(which(apply(B_beta,2,check_function)))
length(which(apply(D_beta,2,check_function)))
length(which(apply(LASSO_beta,2,check_function)))

[1] 10

[1] 48

[1] 6

[1] 0

In [9]:
length(which(apply(A_beta,2,check_function2)))
length(which(apply(B_beta,2,check_function2)))
length(which(apply(D_beta,2,check_function2)))
length(which(apply(LASSO_beta,2,check_function2)))

[1] 100

[1] 99

[1] 100

[1] 100

In [6]:
mean(apply(A_beta,2,distance_function_2))
mean(apply(B_beta,2,distance_function_2))
mean(apply(D_beta,2,distance_function_2))
mean(apply(LASSO_beta,2,distance_function_2))

[1] 0.06325886

[1] 0.06325276

[1] 0.06467658

[1] 0.06354467

In [16]:
length(which(apply(A_beta_11,2,check_function_2)))
length(which(apply(D_beta_11,2,check_function_2)))
length(which(apply(L_beta_11,2,check_function_2)))

[1] 94

[1] 96

[1] 100

In [18]:
mean(apply(A_beta_11,2,distance_function))
mean(apply(A_beta_11,2,distance_function_2))

[1] 0.1206888

[1] 0.0746402

In [19]:
mean(apply(D_beta_11,2,distance_function))
mean(apply(D_beta_11,2,distance_function_2))

[1] 0.1177523

[1] 0.07711446

In [20]:
mean(apply(L_beta_11,2,distance_function))
mean(apply(L_beta_11,2,distance_function_2))

[1] 0.09590429

[1] 0.06800268